# Bug triage with Deep Learning - Fasttext

In [1]:
from __future__ import print_function, division

In [2]:
import fasttext
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [3]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

Using TensorFlow backend.


## Configurações Globais

In [4]:
MAX_SEQUENCE_LENGTH_T = 100 # 40
MAX_SEQUENCE_LENGTH_D = 500 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

### Parse bugs preproprecessed

In [5]:
# Domain to use
DOMAIN = 'netbeans'
METHOD = 'fasttext'
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = 'propose_feature@number_of_epochs@epochs_64batch({})'.format(DOMAIN)
SAVE_PATH_FEATURE = 'propose_feature_@number_of_epochs@epochs_64batch({})'.format(DOMAIN)

# Test bugs
PATH_TEST = 'data/processed/{}/test.txt'.format(DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

## Auxiliary methods

In [6]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [7]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [8]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


216715

### Dicionário de títulos e descrições

In [9]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 1min 28s, sys: 8.83 s, total: 1min 36s
Wall time: 2min 5s


#### Creating buckets and initialize the retrieval method

#### Get buckets for each issue

In [10]:
verbose = 0
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare dataset

In [ ]:
experiment.prepare_dataset(issues_by_buckets)

Reading train data


#### Creating the queries to test

In [ ]:
# Read and create the test queries duplicate
experiment.create_queries()

#### Bugs train

In [ ]:
bugs_train_ids = []
for bugs in experiment.baseline.train_data:
    bugs_train_ids.append(bugs[0])
    bugs_train_ids.append(bugs[1])

#### Corpus

In [ ]:
lines_title = [baseline.bug_set[bug_id]['title'] for bug_id in bugs_train_ids]
lines_desc = [baseline.bug_set[bug_id]['description'] for bug_id in bugs_train_ids]

lines = []
for title, desc in zip(lines_title, lines_desc):
    lines.append(title)
    lines.append(desc)

In [ ]:
len(lines)

#### Exporting corpus to .txt

In [ ]:
EXPORT_PATH = os.path.join(DIR, 'corpus_fasttext_train.txt')
print(EXPORT_PATH)
with open(EXPORT_PATH, 'w') as f:
    for row in lines:
        f.write("{}\n".format(str(row).encode("utf-8")))

#### Train the method

In [ ]:
%%time

import os

cpu = os.cpu_count() - 1

# Skipgram model :
model = fasttext.train_unsupervised(EXPORT_PATH, 
                                    model='cbow', 
                                    minCount=1,
                                    neg=10,
                                    wordNgrams=3,
                                    epoch=15, 
                                    dim=300, 
                                    loss='hs', 
                                    thread=cpu, 
                                    verbose=2)

#### Rank@25

In [ ]:
recall, exported_rank = experiment.evaluate_validation_test(retrieval, verbose, model, issues_by_buckets, method=METHOD)
"recall@25 last epoch:", recall

### Retrieval evaluation

In [ ]:
print("Total of queries:", len(retrieval.test))

In [ ]:
exported_rank[:20]

#### Exporting the result

In [ ]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))

In [ ]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [ ]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report